<a href="https://colab.research.google.com/github/charlo1998/terrain-following-simulation/blob/main/ALE_Framework_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if 'google.colab' in sys.modules:
    !rm -rf *
    !git clone https://github.com/Tinynja/Sarsa-phi-EB
    !mv Sarsa-phi-EB/* .
    !rm -rf Sarsa-phi-EB
    # DON'T install packages defined in Pipfile_colab_remove
    !sed -ri "/$(tr '\n' '|' < Pipfile_colab_remove)/d" Pipfile
else:
    print('Skipping GitHub cloning since not running in Colab.')

Cloning into 'Sarsa-phi-EB'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 193 (delta 39), reused 148 (delta 17), pack-reused 0
Receiving objects: 100% (193/193), 613.07 KiB | 2.42 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [2]:
# Install required dependencies
if 'google.colab' in sys.modules:
    # Colab doesn't support pipenv, hence we convert Pipfile into requirements.txt
    !pip install pipenv 1> /dev/null
    !pipenv lock -r > requirements.txt
    !pip install -r requirements.txt 1> /dev/null
else:
    !pipenv lock 1> /dev/null
    !pipenv install --deploy 1> /dev/null

Creating a virtualenv for this project...
Pipfile: /content/Pipfile
Using /usr/local/bin/python (3.7.12) to create virtualenv...
⠼ Creating virtual environment...created virtual environment CPython3.7.12.final.0-64 in 949ms
  creator CPython3Posix(dest=/root/.local/share/virtualenvs/content-cQIIIOO2, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.3.1, setuptools==58.3.0, wheel==0.37.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator

✔ Successfully created virtual environment! 
Virtualenv location: /root/.local/share/virtualenvs/content-cQIIIOO2
Pipfile.lock not found, creating...
Locking [dev-packages] dependencies...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (44ce9b)!
ERROR: pip's 

In [3]:
# Import all supported ROMs into ALE
!ale-import-roms ROMS

[SUPPORTED]                    qbert        ROMS/Q. Bert (1983).bin
[SUPPORTED]                   casino ROMS/Casino - Poker Plus (Paddle) (1979).bin
[SUPPORTED]             demon_attack ROMS/Demon Attack (Death from Above) (1982).bin
[SUPPORTED]                  bowling        ROMS/Bowling (1979).bin
[SUPPORTED]                 seaquest       ROMS/Seaquest (1983).bin
[SUPPORTED]          chopper_command ROMS/Chopper Command (1982).bin
[SUPPORTED]               time_pilot     ROMS/Time Pilot (1983).bin
[SUPPORTED]                 trondead ROMS/TRON - Deadly Discs (TRON Joystick) (1983).bin
[SUPPORTED]                jamesbond ROMS/James Bond 007 (James Bond Agent 007) (1984).bin
[SUPPORTED]                   pacman        ROMS/Pac-Man (1982).bin
[SUPPORTED]                   kaboom ROMS/Kaboom! (Paddle) (1981).bin
[SUPPORTED]                  asterix ROMS/Asterix (AKA Taz) (1983).bin
[SUPPORTED]                    qbert         ROMS/Q-bert (1987).bin
[SUPPORTED]                  pitfal

In [4]:
# Built-in libraries
import re
import sys
import base64
import random
from pathlib import Path

# Pipy libraries
from IPython import display as ipythondisplay
from ale_py import ALEInterface, SDL_SUPPORT
from ale_py.roms import Breakout
from PIL import Image

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [50]:
class EnvALE:
    def __init__(self, rom, out_dir='ale-results', display=False, seed=0):
        self.out_dir = Path(out_dir).resolve()
        self.out_dir.mkdir(exist_ok=True)
    
        self.ale = ALEInterface()
        self.ale.setInt("random_seed", seed)
        if SDL_SUPPORT and 'google.colab' not in sys.modules:
            ale.setBool("sound", True)
            ale.setBool("display_screen", True)
        self.ale.loadROM(rom)

        self.action_space = self.ale.getMinimalActionSet()

        # Default values
        self._timestep = 0
        self._do_record = False
        self._record_padding = None

    def reset(self, do_record=False):
        self.ale.reset_game()
        observation = self._observe()
        self._timestep = 0

        self._do_record = do_record
        self._handle_recording()
        
        return observation
        
    def step(self, action):
        reward = self.ale.act(action)
        observation = self._observe()
        done = self.ale.game_over()
        self._timestep += 1
        
        self._handle_recording()
        
        return observation, reward, done, None

    def show_video(self, scale=1):
        """Show a .mp4 video in html format of the recorded episode"""
        filepath = self.out_dir.joinpath('record.mp4')
        video_b64 = base64.b64encode(filepath.read_bytes())
        html = f'''<video alt="{filepath}" autoplay loop controls style="height:300px">
                        <source src="data:video/mp4;base64,{video_b64.decode('ascii')}" type="video/mp4" />
                   </video>'''
        ipythondisplay.display(ipythondisplay.HTML(data=html))
        
    def _observe(self):
        observation = self.ale.getScreenRGB()
        return observation
    
    def _handle_recording(self):
        # Do nothing if not asked to record
        if not self._do_record: return
        # This is a new episode, delete previously recorded steps
        if not self._timestep:
            self.out_dir.joinpath('record').mkdir(exist_ok=True)
            for step_png in self.out_dir.glob('record/step_*.png'):
                step_png.unlink()
            self._record_padding = None
        # Record current timestep png
        out_path = self.out_dir.joinpath(f'record/step_{self._timestep}.png')
        self.ale.saveScreenPNG(str(out_path))
        # Once the episode is over, format all png filenames to have the same integer 0 padding
        if self.ale.game_over():
            self._record_padding = len(str(self._timestep))
            self._standardize_record_padding()
            self._png_to_mp4()
    
    def _standardize_record_padding(self):
        number_pattern = re.compile('\d+')
        for png in self.out_dir.glob('record/step_*.png'):
            timestep = int(number_pattern.search(png.stem).group(0))
            new_name = png.parent.joinpath(f'step_{timestep:0{self._record_padding}d}.png')
            png.rename(new_name)

    def _png_to_mp4(self):
        """Convert the recorded set of png files into a mp4 video"""
        in_dir = self.out_dir.joinpath('record')
        in_pattern = self.out_dir.joinpath(f'record/step_%0{self._record_padding}d.png')
        out_file = self.out_dir.joinpath('record.mp4')
        !cd $in_dir; ffmpeg -hide_banner -loglevel error -r 60 -i $in_pattern -vcodec libx264 -crf 25 -pix_fmt yuv420p -y $out_file

In [51]:
env = EnvALE(Breakout)

# Play an episode
done, observation = False, env.reset(do_record=True)
while not done:
    action = random.choice(env.action_space)
    observation, reward, done, info = env.step(action)
env.show_video()

In [60]:
class BreakoutEnv(EnvALE):
  def step(self, action):
    if action == 0:
        action = env.action_space[0]
    elif action == 1:
        action = env.action_space[1]
    elif action == 2:
        action = env.action_space[2]
    elif action == 3:
        action = env.action_space[3]
    else:
      print('action not recognised, use 0, 1, 2, or 3')
    reward = self.ale.act(action)
    observation = self._observe()
    done = self.ale.game_over()
    self._timestep += 1
    
    self._handle_recording()
    
    return observation, reward, done, None

  def _observe(self):
      observation = self.ale.getScreenRGB()
      return observation.flatten()
    

In [61]:
env = BreakoutEnv(Breakout)

# Play an episode
done, observation = False, env.reset(do_record=True)
while not done:
    #action = random.choice(env.action_space)
    action = random.choice(range(4))
    observation, reward, done, info = env.step(action)
env.show_video()

In [66]:
env._observe()

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

Sarsa implementation


In [10]:
# install dependencies
!pip install torch torchvision pyvirtualdisplay matplotlib seaborn pandas numpy pathlib gym
!sudo apt-get install xvfb

# Run this cell

# type hinting 
from typing import Sequence, Tuple, Dict, Any, Optional

import numpy as np

# torch stuff
#import torch
#import torch.nn as nn
#import torch.nn.functional as F 
#from torch import optim

# env
import gym

# data manipulation, colab dispaly, and plotting
import pandas as pd
import matplotlib.pyplot as plt


# misc util
import random

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [67]:
def run_episodes(env, Q, num_episodes=100, to_print=False):
    '''
    Run some episodes to test the policy
    '''
    tot_rew = []
    state = env.reset()

    for _ in range(num_episodes):
        done = False
        game_rew = 0

        while not done:
            # select a greedy action
            next_state, rew, done, _ = env.step(greedy(Q, state))

            state = next_state
            game_rew += rew 
            if done:
                state = env.reset()
                tot_rew.append(game_rew)
    return tot_rew

#eps-greedy
def eps_greedy(Q, s, eps=0.1):
    '''
    Epsilon greedy policy
    '''
    if np.random.uniform(0,1) < eps:
        # Choose a random action
        return np.random.randint(Q.shape[1])
    else:
        # Choose the action of a greedy policy
        return greedy(Q, s)

#formatting action for ale env
def formatAction(action):
  return action

#sarsa
def SARSA(env, lr=0.01, num_episodes=10000, eps=0.3, gamma=0.95, eps_decay=0.00005):

    nA = len(env.action_space)
    nS = len(env._observe())

    # Initialize the Q matrix
    # Q: matrix nS*nA where each row represent a state and each colums represent a different action
    Q = np.zeros((nS, nA))
    games_reward = []
    test_rewards = []

    for ep in range(num_episodes):
        state = env.reset()
        done = False
        tot_rew = 0

        # decay the epsilon value until it reaches the threshold of 0.01
        if eps > 0.01:
            eps -= eps_decay


        action = eps_greedy(Q, state, eps)
        move = formatAction(action)
        

        # loop the main body until the environment stops
        while not done:
            next_state, rew, done, _ = env.step(move) # Take one step in the environment

            # choose the next action (needed for the SARSA update)
            next_action = eps_greedy(Q, next_state, eps) 
            # SARSA update
            Q[state][action] = Q[state][action] + lr*(rew + gamma*Q[next_state][next_action] - Q[state][action])

            state = next_state
            action = next_action
            move = formatAction(action)
            tot_rew += rew
            if done:
                games_reward.append(tot_rew)

        # Test the policy every 300 episodes and print the results
        if (ep % 300) == 0:
            test_rew = run_episodes(env, Q, 1000)
            print("Episode:{:5d}  Eps:{:2.4f}  Rew:{:2.4f}".format(ep, eps, test_rew))
            test_rewards.append(test_rew)

    return Q